In [10]:
import sys
sys.version

'3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]'

In [11]:
import selenium
selenium.__version__

'3.141.0'

In [3]:
import pandas as pd
import re
import openpyxl
import win32com.client

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta

driver = webdriver.Chrome()
driver.get("https://report.merimen.com.my/claims/index.cfm")

username = driver.find_element("id","sleUserName")
password = driver.find_element("id","slePassword")
login = driver.find_element("xpath","//input[@value='Login']")
username.send_keys("MIGAMINUR")
password.send_keys("R10.5.2004n")
login.click()

clm_types_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//input[@value='ClmTypes']"))
)
clm_types_button.click()
select_all_element = driver.find_element("xpath","//span[contains(text(), '[Select All]')]")
select_all_element.click()
checkbox_element = driver.find_element(By.XPATH, "//td[input[@type='CHECKBOX' and @clmcls='Motor' and contains(@value, '4096')]]/input[@type='CHECKBOX']")
checkbox_element.click()
done_button = driver.find_element(By.XPATH, "//td[input[@value='Done'and contains(@value, 'Done')]]/input[@value='Done']")
done_button.click()

click_here = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//a[text()='Report on Outstanding TPBI Claims']"))
)
click_here.click()

date_choice = input("Enter your choice:\n"
                    "1: Last Year\n"
                    "2: Current Year\n"
                    "3: Last Month\n"
                    "4: Current Month\n"
                    "5: Last Week\n"
                    "6: Current Week\n"
                    "7: Yesterday\n"
                    "8: Today\n"
                    "9: Manually Input\n")

if date_choice == '1':
    today = datetime.now()
    first_day_of_last_year = today.replace(year=today.year - 1, month=1, day=1)
    last_day_of_last_year = today.replace(year=today.year - 1, month=12, day=31)
    from_date_input = first_day_of_last_year.strftime("%d/%m/%Y")
    to_date_input = last_day_of_last_year.strftime("%d/%m/%Y")
elif date_choice == '2':
    today = datetime.now()
    first_day_of_current_year = today.replace(month=1, day=1)
    last_day_of_current_year = today.replace(month=12, day=31)
    from_date_input = first_day_of_current_year.strftime("%d/%m/%Y")
    to_date_input = last_day_of_current_year.strftime("%d/%m/%Y")
elif date_choice == '3':
    today = datetime.now()
    first_day_of_current_month = today.replace(day=1)
    last_day_of_last_month = first_day_of_current_month - timedelta(days=1)
    last_day_of_last_month = last_day_of_last_month.replace(day=1)
    from_date_input = last_day_of_last_month.strftime("%d/%m/%Y")
    to_date_input = (first_day_of_current_month - timedelta(days=1)).strftime("%d/%m/%Y")
elif date_choice == '4':
    today = datetime.now()
    first_day_of_current_month = today.replace(day=1)
    last_day_of_current_month = today
    from_date_input = first_day_of_current_month.strftime("%d/%m/%Y")
    to_date_input = last_day_of_current_month.strftime("%d/%m/%Y")
elif date_choice == '5':
    today = datetime.now()
    last_week = today - timedelta(weeks=1)
    first_day_of_last_week = last_week - timedelta(days=last_week.weekday())
    from_date_input = first_day_of_last_week.strftime("%d/%m/%Y")
    to_date_input = last_week.strftime("%d/%m/%Y")
elif date_choice == '6':
    today = datetime.now()
    first_day_of_current_week = today - timedelta(days=today.weekday())
    from_date_input = first_day_of_current_week.strftime("%d/%m/%Y")
    to_date_input = today.strftime("%d/%m/%Y")
elif date_choice == '7':
    today = datetime.now()
    yesterday = today - timedelta(days=1)
    from_date_input = yesterday.strftime("%d/%m/%Y")
    to_date_input = yesterday.strftime("%d/%m/%Y")
elif date_choice == '8':
    today = datetime.now()
    from_date_input = today.strftime("%d/%m/%Y")
    to_date_input = today.strftime("%d/%m/%Y")
elif date_choice == '9':
    from_date_input = input("Enter From Date (DD/MM/YYYY): ")
    to_date_input = input("Enter To Date (DD/MM/YYYY): ")


from_date = driver.find_element("id","DrFrom")
to_date = driver.find_element("id","DrTo")
opinion_report = driver.find_element("id","_rbDateBy4")
process_report = driver.find_element("id","td$ClaimMenu$2")
from_date.clear()
from_date.send_keys(from_date_input)
to_date.clear()
to_date.send_keys(to_date_input)

opinion_report.click()
process_report.click()

table = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "//table[@border='1']"))
)
rows = table.find_elements(By.TAG_NAME, "tr")

no_values = []
claim_numbers = []
liable_amounts = []
panel_solicitors = []
pics = []
panel_solicitor_assigned_dates = []
latest_solicitor_opinion_report_dates = []

for row in rows[1:]:  
    cells = row.find_elements(By.TAG_NAME, "td")
    
    no_value = cells[0].text
    claim_no = cells[1].text  
    liable_amount = cells[22].text  
    panel_solicitor = cells[8].text
    pic = cells[28].text
    panel_solicitor_assigned_date = cells[12].text
    latest_solicitor_opinion_report_date = cells[18].text
    
    no_values.append(no_value)
    claim_numbers.append(claim_no)
    liable_amounts.append(liable_amount)
    panel_solicitors.append(panel_solicitor)
    pics.append(pic)
    panel_solicitor_assigned_dates.append(panel_solicitor_assigned_date)
    latest_solicitor_opinion_report_dates.append(latest_solicitor_opinion_report_date)
    

ddata = {
    'No': no_values,
    'Claim No': claim_numbers,
    'Solicitor Worksheet Liable Amount': liable_amounts,
    'Panel Solicitor': panel_solicitors,
    'PIC': pics,
    'Panel Solicitor Assigned Date': panel_solicitor_assigned_dates,
    'Latest Solicitor Opinion Report Submitted Date': latest_solicitor_opinion_report_dates
}

df = pd.DataFrame(ddata)
df['Solicitor Worksheet Liable Amount'] = df['Solicitor Worksheet Liable Amount'].str.replace('[^\d.]', '', regex=True)
df['Solicitor Worksheet Liable Amount'] = pd.to_numeric(df['Solicitor Worksheet Liable Amount'], errors='coerce')
df['Claim No'] = df['Claim No'].str.replace(r'-.+', '', regex=True)

grouped_df = df.groupby(['Claim No', 'Latest Solicitor Opinion Report Submitted Date']).agg({
    'Solicitor Worksheet Liable Amount': 'sum',
    'No': 'first',
    'Panel Solicitor': 'first',
    'PIC': 'first',
    'Panel Solicitor Assigned Date': 'first',
}).reset_index()

column_order = ['No', 'Claim No', 'Solicitor Worksheet Liable Amount', 'Panel Solicitor', 'PIC', 'Panel Solicitor Assigned Date', 'Latest Solicitor Opinion Report Submitted Date']
grouped_df = grouped_df[column_order]
grouped_df['No'] = range(1, len(grouped_df) + 1)

output_file = 'Opinion_reserve.xlsx'
grouped_df.to_excel(output_file, index=False)

print(f'Data saved to {output_file}')


opinion_reserve_file = 'Opinion_reserve.xlsx'
opinion_reserve_wb = openpyxl.load_workbook(opinion_reserve_file)
opinion_reserve_ws = opinion_reserve_wb.active

tpbi_update_file = 'TPBI UPDATE RESERVE MACRO.xlsm'
tpbi_update_wb = openpyxl.load_workbook(tpbi_update_file, data_only=False) 
tpbi_update_ws = tpbi_update_wb.active

no_column = opinion_reserve_ws['A']
claim_no_column = opinion_reserve_ws['B']
liable_amount_column = opinion_reserve_ws['C']

for cell in no_column:
    tpbi_update_ws.cell(row=6 + cell.row, column=1, value=cell.value)

for cell in claim_no_column:
    tpbi_update_ws.cell(row=6 + cell.row, column=2, value=cell.value)

for cell in liable_amount_column:
    tpbi_update_ws.cell(row=6 + cell.row, column=7, value=cell.value)

tpbi_update_wb.save(tpbi_update_file)
opinion_reserve_wb.close()
tpbi_update_wb.close()

print(f'Data from "No," "Claim No," and "Solicitor Worksheet Liable Amount" columns transferred to "TPBI UPDATE RESERVE MACRO.xlsm"')

# File path to the TPBI UPDATE RESERVE MACRO Excel file
#excel_file_path = 'path/to/TPBI UPDATE RESERVE MACRO.xlsm'

# Create Excel application object
#excel_app = win32com.client.Dispatch("Excel.Application")

# Make Excel visible (optional, you can set it to False if you want it to run in the background)
#excel_app.Visible = True

# Open the Excel file
#workbook = excel_app.Workbooks.Open(excel_file_path)

# Reference the sheet containing the button
#sheet = workbook.Worksheets("claims")

# Reference the button by its name
#button = sheet.Buttons("CLICK HERE BEFORE CLICK ON INQUIRY BUTTON")

# Click the button
#button.OnAction = "YourMacroName"  # Replace "YourMacroName" with the actual name of your VBA macro
#button.Click()

# Save the changes (optional, if you want to save any changes made by the macro)
#workbook.Save()

# Close Excel
#excel_app.Quit()

Enter your choice:
1: Last Year
2: Current Year
3: Last Month
4: Current Month
5: Last Week
6: Current Week
7: Yesterday
8: Today
9: Manually Input
5
Data saved to Opinion_reserve.xlsx
Data from "No," "Claim No," and "Solicitor Worksheet Liable Amount" columns transferred to "TPBI UPDATE RESERVE MACRO.xlsm"


In [3]:
#Tester

import pandas as pd
import re
import os
import openpyxl
import win32com.client
import win32com.client as win32

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta

driver = webdriver.Chrome()
driver.get("https://report.merimen.com.my/claims/index.cfm")

username = driver.find_element("id","sleUserName")
password = driver.find_element("id","slePassword")
login = driver.find_element("xpath","//input[@value='Login']")
username.send_keys("MIGAMINUR")
password.send_keys("R10.5.2004n")
login.click()

clm_types_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//input[@value='ClmTypes']"))
)
clm_types_button.click()
select_all_element = driver.find_element("xpath","//span[contains(text(), '[Select All]')]")
select_all_element.click()
checkbox_element = driver.find_element(By.XPATH, "//td[input[@type='CHECKBOX' and @clmcls='Motor' and contains(@value, '4096')]]/input[@type='CHECKBOX']")
checkbox_element.click()
done_button = driver.find_element(By.XPATH, "//td[input[@value='Done'and contains(@value, 'Done')]]/input[@value='Done']")
done_button.click()

click_here = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//a[text()='Report on Outstanding TPBI Claims']"))
)
click_here.click()

date_choice = input("Enter your choice:\n"
                    "1: Last Year\n"
                    "2: Current Year\n"
                    "3: Last Month\n"
                    "4: Current Month\n"
                    "5: Last Week\n"
                    "6: Current Week\n"
                    "7: Yesterday\n"
                    "8: Today\n"
                    "9: Manually Input\n")

if date_choice == '1':
    today = datetime.now()
    first_day_of_last_year = today.replace(year=today.year - 1, month=1, day=1)
    last_day_of_last_year = today.replace(year=today.year - 1, month=12, day=31)
    from_date_input = first_day_of_last_year.strftime("%d/%m/%Y")
    to_date_input = last_day_of_last_year.strftime("%d/%m/%Y")
elif date_choice == '2':
    today = datetime.now()
    first_day_of_current_year = today.replace(month=1, day=1)
    last_day_of_current_year = today.replace(month=12, day=31)
    from_date_input = first_day_of_current_year.strftime("%d/%m/%Y")
    to_date_input = last_day_of_current_year.strftime("%d/%m/%Y")
elif date_choice == '3':
    today = datetime.now()
    first_day_of_current_month = today.replace(day=1)
    last_day_of_last_month = first_day_of_current_month - timedelta(days=1)
    last_day_of_last_month = last_day_of_last_month.replace(day=1)
    from_date_input = last_day_of_last_month.strftime("%d/%m/%Y")
    to_date_input = (first_day_of_current_month - timedelta(days=1)).strftime("%d/%m/%Y")
elif date_choice == '4':
    today = datetime.now()
    first_day_of_current_month = today.replace(day=1)
    last_day_of_current_month = today
    from_date_input = first_day_of_current_month.strftime("%d/%m/%Y")
    to_date_input = last_day_of_current_month.strftime("%d/%m/%Y")
elif date_choice == '5':
    today = datetime.now()
    last_week = today - timedelta(weeks=1)
    first_day_of_last_week = last_week - timedelta(days=last_week.weekday())
    from_date_input = first_day_of_last_week.strftime("%d/%m/%Y")
    to_date_input = last_week.strftime("%d/%m/%Y")
elif date_choice == '6':
    today = datetime.now()
    first_day_of_current_week = today - timedelta(days=today.weekday())
    from_date_input = first_day_of_current_week.strftime("%d/%m/%Y")
    to_date_input = today.strftime("%d/%m/%Y")
elif date_choice == '7':
    today = datetime.now()
    yesterday = today - timedelta(days=1)
    from_date_input = yesterday.strftime("%d/%m/%Y")
    to_date_input = yesterday.strftime("%d/%m/%Y")
elif date_choice == '8':
    today = datetime.now()
    from_date_input = today.strftime("%d/%m/%Y")
    to_date_input = today.strftime("%d/%m/%Y")
elif date_choice == '9':
    from_date_input = input("Enter From Date (DD/MM/YYYY): ")
    to_date_input = input("Enter To Date (DD/MM/YYYY): ")


from_date = driver.find_element("id","DrFrom")
to_date = driver.find_element("id","DrTo")
opinion_report = driver.find_element("id","_rbDateBy4")
process_report = driver.find_element("id","td$ClaimMenu$2")
from_date.clear()
from_date.send_keys(from_date_input)
to_date.clear()
to_date.send_keys(to_date_input)

opinion_report.click()
process_report.click()

table = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "//table[@border='1']"))
)
rows = table.find_elements(By.TAG_NAME, "tr")

no_values = []
claim_numbers = []
liable_amounts = []
panel_solicitors = []
pics = []
panel_solicitor_assigned_dates = []
latest_solicitor_opinion_report_dates = []

for row in rows[1:]:  
    cells = row.find_elements(By.TAG_NAME, "td")
    
    no_value = cells[0].text
    claim_no = cells[1].text  
    liable_amount = cells[22].text  
    panel_solicitor = cells[8].text
    pic = cells[28].text
    panel_solicitor_assigned_date = cells[12].text
    latest_solicitor_opinion_report_date = cells[18].text
    
    no_values.append(no_value)
    claim_numbers.append(claim_no)
    liable_amounts.append(liable_amount)
    panel_solicitors.append(panel_solicitor)
    pics.append(pic)
    panel_solicitor_assigned_dates.append(panel_solicitor_assigned_date)
    latest_solicitor_opinion_report_dates.append(latest_solicitor_opinion_report_date)
    

ddata = {
    'No': no_values,
    'Claim No': claim_numbers,
    'Solicitor Worksheet Liable Amount': liable_amounts,
    'Panel Solicitor': panel_solicitors,
    'PIC': pics,
    'Panel Solicitor Assigned Date': panel_solicitor_assigned_dates,
    'Latest Solicitor Opinion Report Submitted Date': latest_solicitor_opinion_report_dates
}

df = pd.DataFrame(ddata)
df['Solicitor Worksheet Liable Amount'] = df['Solicitor Worksheet Liable Amount'].str.replace('[^\d.]', '', regex=True)
df['Solicitor Worksheet Liable Amount'] = pd.to_numeric(df['Solicitor Worksheet Liable Amount'], errors='coerce')
df['Claim No'] = df['Claim No'].str.replace(r'-.+', '', regex=True)

grouped_df = df.groupby(['Claim No', 'Latest Solicitor Opinion Report Submitted Date']).agg({
    'Solicitor Worksheet Liable Amount': 'sum',
    'No': 'first',
    'Panel Solicitor': 'first',
    'PIC': 'first',
    'Panel Solicitor Assigned Date': 'first',
}).reset_index()

column_order = ['No', 'Claim No', 'Solicitor Worksheet Liable Amount', 'Panel Solicitor', 'PIC', 'Panel Solicitor Assigned Date', 'Latest Solicitor Opinion Report Submitted Date']
grouped_df = grouped_df[column_order]
grouped_df['No'] = range(1, len(grouped_df) + 1)

output_file = 'Opinion_reserve.xlsx'
grouped_df.to_excel(output_file, index=False)

print(f'Data saved to {output_file}')

#change from openpyxl to win32 start here

opinion_reserve_file = r'C:\Users\acer\Desktop\kerja\Code'
tpbi_update_file = r'C:\Users\acer\Desktop\kerja\Code'

# Check if Opinion_reserve.xlsx exists
if not os.path.isfile(opinion_reserve_file):
    print(f"Error: File '{opinion_reserve_file}' not found.")
else:
    # Open Excel instances
    excel_app_opinion_reserve = win32.Dispatch('Excel.Application')
    excel_app_opinion_reserve.Visible = False

    try:
        workbook_opinion_reserve = excel_app_opinion_reserve.Workbooks.Open(opinion_reserve_file)
        sheet_opinion_reserve = workbook_opinion_reserve.ActiveSheet

        excel_app_tpbi_update = win32.Dispatch('Excel.Application')
        excel_app_tpbi_update.Visible = False  
        workbook_tpbi_update = excel_app_tpbi_update.Workbooks.Open(tpbi_update_file)
        sheet_tpbi_update = workbook_tpbi_update.ActiveSheet

        # Transfer data from Opinion_reserve.xlsx to TPBI UPDATE RESERVE MACRO.xlsm
        no_column = sheet_opinion_reserve.Columns(1)
        claim_no_column = sheet_opinion_reserve.Columns(2)
        liable_amount_column = sheet_opinion_reserve.Columns(3)

        for cell in no_column:
            sheet_tpbi_update.Cells(6 + cell.Row, 1).Value = cell.Value

        for cell in claim_no_column:
            sheet_tpbi_update.Cells(6 + cell.Row, 2).Value = cell.Value

        for cell in liable_amount_column:
            sheet_tpbi_update.Cells(6 + cell.Row, 7).Value = cell.Value

        # Save and close Excel instances
        workbook_opinion_reserve.Close(SaveChanges=False)
        workbook_tpbi_update.Close(SaveChanges=True)

        excel_app_opinion_reserve.Quit()
        excel_app_tpbi_update.Quit()

        print(f'Data from "No," "Claim No," and "Solicitor Worksheet Liable Amount" columns transferred to "TPBI UPDATE RESERVE MACRO.xlsm"')

    except Exception as e:
        print(f"An error occurred: {e}")


# File path to the TPBI UPDATE RESERVE MACRO Excel file
#excel_file_path = 'path/to/TPBI UPDATE RESERVE MACRO.xlsm'

# Create Excel application object
#excel_app = win32com.client.Dispatch("Excel.Application")

# Make Excel visible (optional, you can set it to False if you want it to run in the background)
#excel_app.Visible = True

# Open the Excel file
#workbook = excel_app.Workbooks.Open(excel_file_path)

# Reference the sheet containing the button
#sheet = workbook.Worksheets("claims")

# Reference the button by its name
#button = sheet.Buttons("CLICK HERE BEFORE CLICK ON INQUIRY BUTTON")

# Click the button
#button.OnAction = "YourMacroName"  # Replace "YourMacroName" with the actual name of your VBA macro
#button.Click()

# Save the changes (optional, if you want to save any changes made by the macro)
#workbook.Save()

# Close Excel
#excel_app.Quit()

Enter your choice:
1: Last Year
2: Current Year
3: Last Month
4: Current Month
5: Last Week
6: Current Week
7: Yesterday
8: Today
9: Manually Input
6
Data saved to Opinion_reserve.xlsx
Error: File 'C:\Users\acer\Desktop\kerja\Code' not found.
